[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=13%20Julia-Python20-%2002%20Accelerating%20Python%20with%20JuliaCall.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>

# Fall-in-love-with-Julia: Accelerating Python 101

For Julia Python interactions there are two packages:

|    | [PyCall.jl](https://github.com/JuliaPy/PyCall.jl) | [PythonCall.jl](https://github.com/cjdoris/PythonCall.jl) |
| -: | --------- | ------------- |
| pypi | with [`PyJulia`](https://github.com/JuliaPy/pyjulia) python package (simply called `julia` on pypi) | with [`JuliaCall`](https://pypi.org/project/juliacall/) python package |
| conversions | automatically converts between native types | no auto-conversion, just wrapping |
| dependencies | Global package management via `Conda.jl` | Project-separated package management via `CondaPkg.jl` |
| run python | use `py"..."` | use `@pyexec "..."` |

Outline
- [01](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=13%20Julia-Python20-%2001%20Accelerating%20Python%20with%20PyJulia.ipynb) first notebook shows how to use PyJulia
- **[02](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=13%20Julia-Python20-%2002%20Accelerating%20Python%20with%20JuliaCall.ipynb) second notebook is about JuliaCall**
- [03](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=13%20Julia-Python20-%2003%20Use%20Python%20with%20PyCall%20and%20PythonCall.ipynb) using Python from Julia with both PyCall and PythonCall
- [special extra](https://mybinder.org/v2/gh/jolin-io/workshop-accelerate-Python-with-Julia/main?filepath=03-example-cython-vs-cpp-vs-julia.ipynb) - Julia vs C++

# JuliaCall

In [1]:
import os
from juliacall import Main as jl

# JuliaCall comes with its own Julia dependency file juliapkg.json
# however for binder it is much simpler to just reuse binder's installation mechanism
os.environ['PYTHON_JULIAPKG_PROJECT'] = '/path/to/project'
os.environ['PYTHON_JULIAPKG_OFFLINE'] = 'yes'
%load_ext juliacall

In [2]:
import numpy as np
import pandas as pd

array = np.arange(10)
array

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

JuliaCall has no interpolation, however you can add variables as arguments to the multiline %%julia magic. They are then made available on the julia site.

`JuliaCall` only wraps the objects. Conversion happens within function calls.

In [3]:
%%julia array
@show typeof(array)  # @show does work because there is no interpolation
result = isodd.(array)
@show typeof(result)

typeof(array) = PyArray{Int64, 1, true, true, Int64}
typeof(result) = BitVector


BitVector (alias for BitArray{1})

In [4]:
result = %julia isodd.(array) 
type(result)

juliacall.VectorValue

No autoconversion means that you manually need to ensure everything has the right type.

For me the most common pitfall is to work with `True`/`False` values

In [5]:
mycondition = (array == 40).any()
print(type(mycondition))
mycondition

<class 'numpy.bool_'>


False

In [6]:
%%julia mycondition
# always wrap a bool with pytruth 
if pytruth(mycondition)
    println("yes! works")
else
    println("also works :)! (we only want to test that julia actually gets a julia Bool value)")
end

also works :)! (we only want to test that julia actually gets a julia Bool value)


# DataFrames

In [8]:
data = {
  "calories": [420, 380, 390],
  "duration": [50, 40, 45]
}
df = pd.DataFrame(data, index = ["day1", "day2", "day3"])
df

,calories,duration
day1,420,50
day2,380,40
day3,390,45


JuliaCall has special support for pandas tables - they are wrapped so that they can be passed directly to DataFrame

In [9]:
%%julia df
using DataFrames
@show typeof(df)
DataFrame(df)

typeof(df) = PyPandasDataFrame


,calories,duration
,Int64,Int64
1,420,50
2,380,40
3,390,45


appreciate the lovely html rendering of a julia DataFrame :D

The other way around is only slightly more complicated

In [10]:
%%julia
df2 = DataFrame(grp=repeat(1:2, 3), x=6:-1:1, y=4:9, z=[3:7; missing], id='a':'f')

,grp,x,y,z,id
,Int64,Int64,Int64,Int64?,Char
1,1,6,4,3,a
2,2,5,5,4,b
3,1,4,6,5,c
4,2,3,7,6,d
5,1,2,8,7,e
6,2,1,9,missing,f


In [11]:
pd.DataFrame(jl.pytable(jl.df2))

,grp,x,y,z,id
0,1,6,4,3,a
1,2,5,5,4,b
2,1,4,6,5,c
3,2,3,7,6,d
4,1,2,8,7,e
5,2,1,9,missing,f


# Next: [03 PyCall](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=13%20Julia-Python20-%2003%20Use%20Python%20with%20PyCall%20and%20PythonCall.ipynb)


For questions or suggestions please contact me at stephan.sahm@jolin.io

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>
